In [1]:
# Imports

from dotenv import load_dotenv
from openai import OpenAI, AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import pandas as pd
from datetime import datetime
import pypandoc
pypandoc.download_pandoc()


In [2]:
# Load environment variables

load_dotenv(override=True)
openai = OpenAI()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [3]:
# System prompt for writter

writter_system_prompt = """You are writing a series of articles as study guides for topics related to the AWS Advanced Networking Exam. The course guide can be found here: https://d1.awsstatic.com/onedam/marketing-channels/website/aws/en_US/certification/approved/pdfs/docs-advnetworking-spec/AWS-Certified-Advanced-Networking-Specialty_Exam-Guide.pdf.

Please draft an article in the style of Bit the Chipmunk. Bit the Chipmunk is a cute, furry, and friendly woodland creature who also happens to be an AWS expert. He from time to time uses language and expressions that remind us that he is a chipmunk, for example by using nut or forest related metaphors or examples. However, do not make more than 5 chipmunk references in an entire article, and keep these references limitted to the introduction paragraph and the concluding paragraph -- do not include these references anywhere else.

Use your own knowledge and search AWS documentation, blogs, and white papers as sources for the content.

Include a Further Reading section with useful links related to the content of the article that you generate. Also, please include a brief introduction to the topic at the start of the article. Keep the tone light and friendly -- avoid matching the style of the source documents. Be sure to include exam scenarios and exam traps."""


In [ ]:
# Define generate_article function

def generate_article(topic: str) -> str:
    """Use OpenAI to generate the article."""
    prompt = [{"role": "system", "content": writter_system_prompt}] + [{"role": "user", "content": f"Write an article for {topic}."}]
    # response = openai.chat.completions.create(model="gpt-4o-mini", messages=prompt, tools={"web":{}})
    response = openai.responses.create(
    model="gpt-4o-mini",
    # model="gpt-5",
    input=prompt,
    tools=[{
        "type": "web_search"
    }],
)
    return response.output_text


In [5]:
# Create a Pydantic model for the Evaluation
# Pydantic is a powerful Python library used for data validation and data parsing — it makes sure that the data you’re working with is the right type, structure, and format.
# It’s most famous for being the backbone of frameworks like FastAPI, where it automatically validates incoming API data.

from pydantic import BaseModel

class Revision(BaseModel):
    no_more_revision: bool
    feedback: str

In [6]:
# System prompt for reviewer

reviewer_system_prompt = """You are an expert in the AWS Advanced Networking. You will evaluate an article written by the writter agent about a topic from the AWS Advanced Networking Exam.

The agent is given the following instructions:
"writing a series of articles as study guides for topics related to the AWS Advanced Networking Exam. The course guide can be found here: https://d1.awsstatic.com/onedam/marketing-channels/website/aws/en_US/certification/approved/pdfs/docs-advnetworking-spec/AWS-Certified-Advanced-Networking-Specialty_Exam-Guide.pdf.

Please draft an article in the style of Bit the Chipmunk. Bit the Chipmunk is a cute, furry, and friendly woodland creature who also happens to be an AWS expert. He from time to time uses language and expressions that remind us that he is a chipmunk, for example by using nut or forest related metaphors or examples. However, do not make more than 5 chipmunk references in an entire article, and keep these references limitted to the introduction paragraph and the concluding paragraph -- do not include these references anywhere else.

Use your own knowledge and search AWS documentation, blogs, and white papers as sources for the content.

Include a Further Reading section with useful links related to the content of the article that you generate. Also, please include a brief introduction to the topic at the start of the article. Keep the tone light and friendly -- avoid matching the style of the source documents. Be sure to include exam scenarios and exam traps."

Please evaluate the article written by the agent for accuracy and completeness. By accuracy, I mean "does it contain correct information". By completeness, I mean "does it miss any important cases or topics?"
Make sure that the new article's style is in alignment with other articles which are already posted on https://www.bits-guides.com/
If the article is good enough to publish, set no_more_revision as true. Otherwise, set no_more_revision as false and provide your reasoning about why it needs more revision.

Respond only in valid JSON format with the following fields without extra text:
{
    "no_more_revision": true or false,
    "feedback": "your reasoning"
}
Use \\n for newlines inside strings.
"""


In [ ]:
# Define review_article function
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"
# model_name = "gemini-2.5-pro"

import json, re

def review_article(article: str) -> Revision:
    """Use Gemini to review the OpenAI-generated article."""
    prompt = [{"role": "system", "content": reviewer_system_prompt}] + [{"role": "user", "content": f"\n\nArticle:\n{article}."}]
    response = gemini.chat.completions.create(model=model_name, messages=prompt)
    result = response.choices[0].message.content

    # Convert the type of feedback from string to json dict
    # # Try to find a JSON block in the string
    match = re.search(r'\{.*\}', result, re.DOTALL)
    if match:
        json_text = match.group(0)
        result_json = json.loads(json_text)
    else:
        print("No valid JSON found in feedback.")
    
    return result_json

In [9]:
# Define revise_article function

def revise_article(article, feedback) -> str:
    """Use OpenAI to revise the article based on Gemini's feedback."""
    prompt = [{"role": "system", "content": writter_system_prompt}] + [{"role": "user", "content": f"Revise the article based on the feedback provided.\n\nArticle:\n{article}\n\nFeedback:\n{feedback["feedback"]}"}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=prompt)
    return response.choices[0].message.content

In [10]:
def process_topic(topic):

    accepted_article = None
    article = generate_article(topic)

    for round_num in range(3):  # up to 3 review-revise rounds
        print(f"\n--- Round {round_num + 1}: Review ---\n")
        feedback = review_article(article)
        print(feedback)

        if feedback["no_more_revision"]:
            accepted_article = article
            print("\n✅ Reviewer approved this version.")
            break
        else:
            print("\n✏️ Revising article according to feedback...\n")
            article = revise_article(article, feedback)

    # If no explicit approval, assume final version
    if not accepted_article:
        print("\n=== 🏁 ARTICLE IS NOT ACCEPTED AFTER 3 ROUND===\n")
        status = "Incomplete"
    else:
        print("\n=== 🏁 FINAL ACCEPTED ARTICLE ===\n")
        print(accepted_article)
        status = "Completed"

    return status, accepted_article

In [12]:
# --- Main Workflow ---
if __name__ == "__main__":
# === 1. Load the Excel file ===
    input_file = "topics.xlsx"
    df = pd.read_excel(input_file)

# ✅ Explicitly cast to string/object dtype (fixes the warning)
    df["Processed Date"] = df["Processed Date"].astype("string")
    df["Status"] = df["Status"].astype("string")

# === 2. Process only one topic per run ===
    for i, row in df.iterrows():
        status = str(row["Status"]).strip()
        if status == "Completed":
            continue

        topic = row["Topic"]
        print(f"✅ Processing topic: {topic}")
        status, accepted_article = process_topic(topic)
        df.at[i, "Processed Date"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        df.at[i, "Status"] = status

# === 3. Write processed date and status back to Excel ===
        df.to_excel(input_file, index=False)
        print(f"✅ Processing complete. Results saved to {input_file}")

# === 4. Write the accepted article to Word ===
        output_file = topic + ".docx"
        pypandoc.convert_text(
            accepted_article,
            'docx',
            format='md',
            outputfile=output_file,
            extra_args=['--standalone']
        )

        print(f"✅ Accepted article saved to {output_file}")
        break

    else:
        print("✅ All topics already completed!")

✅ Processing topic: Cross Account Resource Sharing

--- Round 1: Review ---

{'no_more_revision': False, 'feedback': "This is an excellent and comprehensive article that is very close to being ready for publication. The structure, tone, and focus on exam-specific details are all top-notch.\n\nHowever, there is one minor but important factual error that needs correction:\n\nIn the section 'Centralized connectivity: AWS Transit Gateway (TGW) and AWS Cloud WAN', under 'TGW sharing', the article states: 'In Sept 2024, AWS added Security Group Referencing...'. The feature was actually launched in September 2023, not 2024. Please correct this date to avoid confusion. A simple change to 'In late 2023, AWS added...' or simply 'AWS recently added...' would be sufficient.\n\nOnce this small correction is made, the article will be accurate and complete."}

✏️ Revising article according to feedback...


--- Round 2: Review ---

{'no_more_revision': False, 'feedback': "This is a very well-structure

InternalServerError: Error code: 503 - [{'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}]